Задача тематическое моделирование

продолжаем исследование датасета с твитами

Скачиваем датасет (источник): положительные, отрицательные.

или можно через ноутбук

!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

как альтернатива можно скачать данные из Роспотребнадзора
https://zpp.rospotrebnadzor.ru/Forum/Appeals
для этого берём ноутбук parse_rospotrebnadzor.ipynb
устанавливаем количество скачанных страниц больше не 50-сят хотябы 500 и для анализа берём только вопросы так как ответы есть не всегда

что надо сделать
1. объединить в одну выборку (это только для твитов), для роспотребнадзора сформировать датасет из вопросов
2. провести исследование и выявить тематики о которых говорят в твитах (для твитов), а для роспотребнадзора так же выявить тематики о которых люди пишут проанализировать
3. сделать визуализацию кластеров тематик
4. проинтерпритировать получившиеся тематики


In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
import os

In [2]:
colnames = ['id', 'tdate', 'tmane', 'ttext', 'ttype', 'trep', 'trtw', 'tfav', 'tstcount', 'tfoll', 'tfrien', 'listcount']

In [25]:
data_df = pd.concat([pd.read_csv('positive.csv', sep=';', header=None, names=colnames, encoding='utf-8'),
                     pd.read_csv('negative.csv', sep=';', header=None,  names=colnames, encoding='utf-8')],
                     ignore_index=True
                   )

In [4]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226834 entries, 0 to 226833
Data columns (total 12 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   id         226834 non-null  int64 
 1   tdate      226834 non-null  int64 
 2   tmane      226834 non-null  object
 3   ttext      226834 non-null  object
 4   ttype      226834 non-null  int64 
 5   trep       226834 non-null  int64 
 6   trtw       226834 non-null  int64 
 7   tfav       226834 non-null  int64 
 8   tstcount   226834 non-null  int64 
 9   tfoll      226834 non-null  int64 
 10  tfrien     226834 non-null  int64 
 11  listcount  226834 non-null  int64 
dtypes: int64(10), object(2)
memory usage: 20.8+ MB


In [5]:
data_df.head()

,id,tdate,tmane,ttext,ttype,trep,trtw,tfav,tstcount,tfoll,tfrien,listcount
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1


In [6]:
import pymorphy2
from tqdm.notebook import tqdm
from nltk.corpus import stopwords
from multiprocessing import Pool



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chinyaev.av\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [42]:
nltk.download('stopwords')
morph = pymorphy2.MorphAnalyzer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chinyaev.av\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [43]:

stopwords_list = stopwords.words('russian')

In [44]:
words_regex = re.compile('\w+')

def find_words(text, regex = words_regex):
    tokens =  regex.findall(text.lower())
    return [w for w in tokens if w.isalpha() and len(w) >= 3]

def lemmatize(words, lemmer = morph, stopwords = stopwords_list):
    lemmas = [lemmer.parse(w)[0].normal_form for w in words]
    return [w for w in lemmas if not w in stopwords 
            and w.isalpha()]

def preprocess(text):
    return (lemmatize(find_words(text)))

def remove_user(data: pd.DataFrame):
    return list(map(lambda x: re.sub(r"@[\w]*", " ", x), data['ttext'].values))


In [45]:
data_df['ttext'] = remove_user(data_df)

In [46]:
stopwords_list.append('rt')

In [47]:
print(preprocess(data_df.ttext.iloc[0]))

['школотый', 'поверь', 'самый', 'общество', 'профилировать', 'предмет', 'тип']


In [48]:
pre_data = list(tqdm(map(preprocess, data_df['ttext']), total=len(data_df)))

  0%|          | 0/226834 [00:00<?, ?it/s]

In [49]:
import csv
with open(r'pre_data_list.csv', 'w', encoding="utf-8") as fp:
#    for item in pre_data:
#        # write each item on a new line
#        fp.write("%s\n" % item)
#    print('Done')
    w = csv.writer(fp, delimiter =";")
    w.writerows(pre_data)


In [50]:
data_df['preproc_text'] = pre_data

In [51]:
data_df.head()

,id,tdate,tmane,ttext,ttype,trep,trtw,tfav,tstcount,tfoll,tfrien,listcount,preproc_text
0,408906692374446080,1386325927,pleease_shut_up,"хоть я и школота, но поверь, у нас то же сам...",1,0,0,0,7569,62,61,0,"[школотый, поверь, самый, общество, профилиров..."
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2,"[всё, таки, немного, похожий, мальчик, всё, ра..."
2,408906695083954177,1386325927,EvgeshaRe,RT : Ну ты идиотка) я испугалась за тебя!!!,1,0,1,0,1273,26,27,0,"[идиотка, испугаться]"
3,408906695356973056,1386325927,ikonnikova_21,"RT : ""Кто то в углу сидит и погибает от голод...",1,0,1,0,1549,19,17,0,"[угол, сидеть, погибать, голод, ещё, порция, в..."
4,408906761416867842,1386325943,JumpyAlex,"Вот что значит страшилка :D\nНо блин,посмотр...",1,0,0,0,597,16,23,1,"[значит, страшилка, блин, посмотреть, всё, час..."


In [52]:
from gensim.models import *
from gensim import corpora

In [53]:
dictionary = corpora.Dictionary(data_df['preproc_text'])

dictionary.filter_extremes(no_below = 10, no_above = 0.9) # игнорируем слова, которые встречаются реже 10 раз или составляют более 0.9 словаря 
dictionary.save('lenta.dict')

In [54]:
corpus = [dictionary.doc2bow(text) for text in data_df['preproc_text']]
corpora.MmCorpus.serialize('lenta.model', corpus)

In [55]:
%time lda = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=7, chunksize=50, update_every=1, passes=2)

Wall time: 2min 25s


In [56]:
lda.show_topics(num_topics=10, num_words=10, formatted=True)

[(0,
  '0.044*"просто" + 0.043*"блин" + 0.030*"писать" + 0.024*"сидеть" + 0.022*"хотеться" + 0.018*"понимать" + 0.018*"друг" + 0.013*"нравиться" + 0.012*"фильм" + 0.012*"хотя"'),
 (1,
  '0.063*"это" + 0.049*"весь" + 0.030*"вообще" + 0.025*"любить" + 0.019*"жизнь" + 0.018*"самый" + 0.018*"год" + 0.014*"плохо" + 0.014*"утро" + 0.013*"твой"'),
 (2,
  '0.059*"мочь" + 0.051*"сегодня" + 0.046*"очень" + 0.039*"это" + 0.033*"человек" + 0.023*"хороший" + 0.018*"час" + 0.015*"сделать" + 0.014*"ждать" + 0.012*"плохой"'),
 (3,
  '0.074*"ещё" + 0.040*"почему" + 0.031*"свой" + 0.026*"болеть" + 0.021*"говорить" + 0.017*"ночь" + 0.016*"каждый" + 0.013*"нибыть" + 0.012*"остаться" + 0.012*"жалко"'),
 (4,
  '0.138*"всё" + 0.029*"большой" + 0.029*"сказать" + 0.024*"дом" + 0.018*"никто" + 0.016*"стать" + 0.014*"посмотреть" + 0.013*"холодно" + 0.013*"дело" + 0.012*"читать"'),
 (5,
  '0.116*"http" + 0.034*"знать" + 0.026*"делать" + 0.023*"спать" + 0.021*"смотреть" + 0.019*"это" + 0.018*"скучать" + 0.017*"дум

In [57]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

In [58]:
%time vis_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)

C:\Anaconda3\envs\python37\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


Wall time: 17.8 s
